In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

In [3]:
# 페이스북 넷플릭스 마이크로소프트
datas = ['META', 'NFLX', 'MSFT']
# 크롤링 할 날짜 지정  예) '2020-01-01'
start_date = '2021-07-01'
end_date = '2022-12-31'

## 데이터 프레임의 달러를 원화로 전처리 하기 위한 코드
# 현재 환율 정보를 가져온다
# 웹사이트에서 환율 정보 스크래핑
url = 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page=1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
table = soup.select_one('td.num')
# 환율 str을 float으로 변환하기 위한 전처리 (,)를 없애줌
exchange = float(table.text.replace(',', ''))
# 현재 시각의 달러 환율 정보를 가져온다.
print('현재', datetime.datetime.now(), f'기준 환율 {exchange}원')
# apply를 위한 달러 => 원화 함수 exchanged 정의
def exchanged(x):
    return round(exchange * x)

# 리스트 datas 안에 들어있는 기업들의 정보를 크롤링 => csv화
for data in datas:
    # fi_data변수에 저장
    
    # 날짜 지정
    fi_data = yf.download(data, start=start_date, end=end_date)
    # 전처리
    ## 필요없는 컬럼 삭제  
    fi_data.drop('Adj Close', axis=1, inplace=True)
    ## 컬럼명 변경
    fi_data.index.name = '날짜'
    fi_data.columns = ['시가', '고가', '저가', '종가', '거래량']
    ## 컬럼 순서 변경
    fi_data = fi_data.iloc[:, [3, 0, 1, 2, 4]]
    ## 달러 => 원화로 변경 (달러가 적힌 컬럼은 종가, 시가, 고가, 저가)
    # 서브셋 정의
    subset = fi_data[['종가', '시가', '고가', '저가']]
    # apply와 exchanged 함수로 달러를 원화로 변경
    subset = subset.apply(exchanged)
    # subset을 df에 적용
    fi_data[['종가', '시가', '고가', '저가']] = subset
    
    # 데이터프레임을 csv 파일로 저장
    fi_data.to_csv(f'./csvs/{data}.csv', index=True)
    print(f'{data}.csv 파일을 저장했습니다.')

현재 2023-03-07 14:10:07.039737 기준 환율 1298.5원
[*********************100%***********************]  1 of 1 completed
META.csv 파일을 저장했습니다.
[*********************100%***********************]  1 of 1 completed
NFLX.csv 파일을 저장했습니다.
[*********************100%***********************]  1 of 1 completed
MSFT.csv 파일을 저장했습니다.


In [2]:
netflix_df = pd.read_csv('./csvs/NFLX.csv')
netflix_df

,날짜,종가,시가,고가,저가,거래량
0,2021-07-01,692802.0,682647.0,697346.0,682647.0,2805400
1,2021-07-02,693373.0,695347.0,699294.0,687413.0,1975500
2,2021-07-06,703320.0,692100.0,704904.0,692100.0,2775100
3,2021-07-07,695944.0,706696.0,707215.0,690360.0,2722500
4,2021-07-08,689192.0,689413.0,695347.0,687023.0,3269000
...,...,...,...,...,...,...
374,2022-12-23,383006.0,384590.0,387550.0,379045.0,4251100
375,2022-12-27,368995.0,380707.0,381201.0,366346.0,5778100
376,2022-12-28,359529.0,366073.0,370319.0,355023.0,5964400
377,2022-12-29,378019.0,367709.0,383707.0,364891.0,9588500
